In [136]:
import sklearn as sk
import numpy as np 
import pandas as pd 

In [137]:
# read in our training data and take the first column as the index column
fraud_train = pd.read_csv('fraudTrain.csv', index_col=0)

# split the training data into x and y
X = fraud_train.drop(columns='is_fraud')
y = fraud_train['is_fraud']


In [138]:
# first look at our data
X.info()
X.sample(3, random_state=1)


<class 'pandas.core.frame.DataFrame'>
Index: 1296675 entries, 0 to 1296674
Data columns (total 21 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   trans_date_trans_time  1296675 non-null  object 
 1   cc_num                 1296675 non-null  int64  
 2   merchant               1296675 non-null  object 
 3   category               1296675 non-null  object 
 4   amt                    1296675 non-null  float64
 5   first                  1296675 non-null  object 
 6   last                   1296675 non-null  object 
 7   gender                 1296675 non-null  object 
 8   street                 1296675 non-null  object 
 9   city                   1296675 non-null  object 
 10  state                  1296675 non-null  object 
 11  zip                    1296675 non-null  int64  
 12  lat                    1296675 non-null  float64
 13  long                   1296675 non-null  float64
 14  city_pop               

,trans_date_trans_time,cc_num,merchant,category,amt,first,last,gender,street,city,state,zip,lat,long,city_pop,job,dob,trans_num,unix_time,merch_lat,merch_long
94102,2019-02-25 08:24:40,374497717543058,fraud_Funk Group,grocery_net,20.00,Linda,Hurst,F,31701 Tucker Square Suite 893,Wilton,ND,58579,47.1709,-100.7944,1190,"Designer, ceramics/pottery",1948-06-30,1595dec12f6f19ceaae9b7df0f8af5c0,1330158280,46.398331,-99.813959
198791,2019-04-12 19:50:15,4428154703770710,"fraud_Prosacco, Kreiger and Kovacek",home,284.88,Brittany,Guerra,F,79209 Gary Dale,Colton,WA,99113,46.5901,-117.1692,761,Chief Marketing Officer,1943-06-30,0ed26b649ed0fce94d8e632b7208dea0,1334260215,45.687331,-117.488135
1238587,2020-05-31 21:50:53,213148039875802,"fraud_Langworth, Boehm and Gulgowski",shopping_net,5.07,Jill,Jacobs,F,034 Kimberly Mountains,Brandon,FL,33510,27.9551,-82.2966,79613,Environmental consultant,1978-11-30,7096316ec1a4b261e8613013827abae7,1370037053,27.254081,-81.974799


# Preprocessing

## 1. Finding and Handling Missing Values

In [139]:
# what columns have null values?
sum(X.isnull().sum()) + y.isnull().sum()

0

In [140]:
# what columns have missing/na values?
sum(X.isna().sum()) + y.isna().sum()

0

No handling of missing values is required as there are no missing values in our trainng dataset

## 2. Finding and Removing Outliers

Removing outliers is an important part of preprocessing as it can:
- distort data analysis 
- reduce machine learning model accuracy and generalization
- impact visual data, skewing the scale 
- more?

For numeric columns (those with an integer or float data type), I chose to use the [Interquartile Range Method](https://online.stat.psu.edu/stat200/lesson/3/3.2) of finding and removing outliers. 

Categorical, or 

## FIXME

In [141]:
# find the numerical columns
num_cols = X.select_dtypes(include=['float64', 'int64']).columns
num_cols

# calculate the IQR lower and upper bounds for each numerical column
def iqr_bounds(col):
    Q1 = col.quantile(0.25)
    Q3 = col.quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5 * IQR
    upper_bound = Q3 + 1.5 * IQR
    return lower_bound, upper_bound

In [142]:
# lets look at the lower and upper bounds for each numerical column
for col in num_cols:
    lower, upper = iqr_bounds(X[col])
    print(f"{col}: lower={lower}, upper={upper}")

cc_num: lower=-6513275846701038.0, upper=1.133557426847813e+16
amt: lower=-100.58499999999998, upper=193.375
zip: lower=-42470.5, upper=140749.5
lat: lower=23.640650000000004, upper=52.920249999999996
long: lower=-121.75800000000001, upper=-55.198
city_pop: lower=-28634.5, upper=49705.5
unix_time: lower=1307798793.0, upper=1390337325.0
merch_lat: lower=23.898184000000008, upper=52.79255199999999
merch_long: lower=-121.88799400000002, upper=-55.24607799999998


In [143]:
# how many outliers are there in each numerical column?
for col in num_cols:
    lower, upper = iqr_bounds(X[col])
    outliers = fraud_train[(fraud_train[col] < lower) | (fraud_train[col] > upper)]
    print(f"{col}: {len(outliers)} outliers")

cc_num: 118789 outliers
amt: 67290 outliers
zip: 0 outliers
lat: 4679 outliers
long: 49922 outliers
city_pop: 242674 outliers
unix_time: 0 outliers
merch_lat: 4967 outliers
merch_long: 41994 outliers


I won't be dropping the rows with outlier values before training a model on the dataset, as I'm interested in comparing how the model performs with and without the outlier values.

### 3. Feature Selection & Feature Engineering

Before training a model, I want to get a good idea of our data. I ask myself questions such as *"is this information redundnat?"* or *"can this data generalized or are all values unique?"*. If the answer to either question is yes, it is usually best to remove the column. However, sometimes there is information we can extract from the column that may not be redundant and can be generalized or categorized. 


One example in our data is the trans_time_trans_date column. At the moment, this column may not help us detect fraud very well since almost every entry is unqiue (as shown below), but there is some interesting information we may want to look at within the column. Information such as day of the week or the hour in which the transaction took place could help us identify fraudulent transactions more accurately. Based on our columns printed below, I plan to extract the following information:

## FIX ME:
- cc_num: number of transaction in past **X time** (days? weeks? months?)
- trans_date_trans_time: month, day, and hour of transaction


In [144]:
def unique_percentage(column):
    return (len(X[column].unique())) / len(X[column]) * 100

for column in X.columns:
    print(f'{round(unique_percentage(column),2)}% of the values in {column} are unique.')

98.31% of the values in trans_date_trans_time are unique.
0.08% of the values in cc_num are unique.
0.05% of the values in merchant are unique.
0.0% of the values in category are unique.
4.08% of the values in amt are unique.
0.03% of the values in first are unique.
0.04% of the values in last are unique.
0.0% of the values in gender are unique.
0.08% of the values in street are unique.
0.07% of the values in city are unique.
0.0% of the values in state are unique.
0.07% of the values in zip are unique.
0.07% of the values in lat are unique.
0.07% of the values in long are unique.
0.07% of the values in city_pop are unique.
0.04% of the values in job are unique.
0.07% of the values in dob are unique.
100.0% of the values in trans_num are unique.
98.31% of the values in unix_time are unique.
96.23% of the values in merch_lat are unique.
98.39% of the values in merch_long are unique.


In [145]:
# all columns and data types
X.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1296675 entries, 0 to 1296674
Data columns (total 21 columns):
 #   Column                 Non-Null Count    Dtype  
---  ------                 --------------    -----  
 0   trans_date_trans_time  1296675 non-null  object 
 1   cc_num                 1296675 non-null  int64  
 2   merchant               1296675 non-null  object 
 3   category               1296675 non-null  object 
 4   amt                    1296675 non-null  float64
 5   first                  1296675 non-null  object 
 6   last                   1296675 non-null  object 
 7   gender                 1296675 non-null  object 
 8   street                 1296675 non-null  object 
 9   city                   1296675 non-null  object 
 10  state                  1296675 non-null  object 
 11  zip                    1296675 non-null  int64  
 12  lat                    1296675 non-null  float64
 13  long                   1296675 non-null  float64
 14  city_pop               

In [146]:
# FIXME: below but unix time? fraud might not be happening in the local timezone of the transaction

In [147]:
# FIXME: DUPLICATE CC numbers BEFORE DROPPING them
# count trans in the last 24 hours? week? month?

In [148]:
# transforming 'trans_date_trans_time' into day of week, hour of day, and month
X['trans_date_trans_time'] = pd.to_datetime(X['trans_date_trans_time'])
X['trans_month'] = X['trans_date_trans_time'].dt.month_name()
X['trans_day_of_week'] = X['trans_date_trans_time'].dt.day_name()
X['trans_hour_of_day'] = X['trans_date_trans_time'].dt.hour


X[['trans_month', 'trans_day_of_week', 'trans_hour_of_day']]

,trans_month,trans_day_of_week,trans_hour_of_day
0,January,Tuesday,0
1,January,Tuesday,0
2,January,Tuesday,0
3,January,Tuesday,0
4,January,Tuesday,0
...,...,...,...
1296670,June,Sunday,12
1296671,June,Sunday,12
1296672,June,Sunday,12
1296673,June,Sunday,12


In [149]:
# look at 

Looking at all the object Dtype columns, I am most interested in keeping the gender, state, job, dob, trans_month, and trans_day_of_week. 

I have chosen to omit the following columns:
- cc_num  
- first  
- last  
- street  
- city  
- zip  
- lat
- long
- merch_lat
- merch_long
- trans_num  
- trans_date_trans_time 

I opted to remove the street, zip, and city columns, as I think training on the state will be enough. Removing these columns aslo reduces noise in our data which will increase the accuracy and reliability of our model. If, after we've trained a model and looked at the feature coefficients, the state in which a person lives carries a high probablity of a fraudulent transaction, we can train a new model to take into consideration the mroe in depth locational features. 

Similarly, I am omitting the cc_number, first, last, lat, long, trans_num, merch_lat, merch_long, and trans_date_trans_time as all of these columns introduce noise in our dataset and can reduce the reliability and accuracy of a model trained on it.

In [150]:
# # dropping all columns we are not interested in
# X = X.drop(columns=['cc_num',
#                     'first',
#                     'last',
#                     'street',
#                     'city',
#                     'zip',
#                     'lat',
#                     'long',
#                     'trans_num',
#                     'unix_time',
#                     'merch_lat',
#                     'merch_long',
#                     'trans_date_trans_time'
#                     ])

# X.info()

In [151]:
# # from sklearn.preprocessing import OneHotEncoder
# ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')
# X_ohe = ohe.fit_transform(X.select_dtypes(include=['object']))

### 4. Correlation Analysis

Before we analyze the correlation between all numerical columns, we need to convert categorical features into numerical features without heirarchy. To do this, I will use sci-kit learn's [OneHotEncoder](https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.OneHotEncoder.html).